In [7]:
from lib.framework_genai.GenAILLM import EnumGenAIModelsIdsOpenAI, EnumGenAIPlatforms, GenAILLM
from lib.framework_genai.GenAIMemory import EnumMemoryType, GenAIMemory
from lib.framework_genai.LoggingAndTelemetry import EnumLogs
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
ai_prefix = "Assistant"
human_prefix = "Human"
verbose_level:EnumLogs = EnumLogs.LOG_LEVEL_DEBUG

gen_ai_llm = GenAILLM(
    platform = EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id = EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO_16k,
    parameters_inference = {
        'max_tokens': 1024, 
        "temperature": 0.1,
        "top_p": 0.2
    },
    ai_prefix = ai_prefix,
    human_prefix = human_prefix,
    component_memory=GenAIMemory(EnumMemoryType.MEMORY_CHAT_BUFFER, ai_prefix, human_prefix),
    platform_configuration = {
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]
    },
    verify_ssl=True,
    verbose_level=verbose_level
)

2024-02-14 17:45:57.970213-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO_16k construido


In [9]:
import xml.etree.ElementTree as ET

def save_part(xml_elements, name, path="./temp/dtsx_query/parts_basic01"):
    if not isinstance(xml_elements, list):
        xml_elements = [xml_elements]

    if "." not in name:
        name = f"{name}.txt"

    xml_str = ""
    for xml_element in xml_elements:
        xml_str += ET.tostring(xml_element, encoding='unicode').strip()
    with open(f"{path}/{name}", "w") as f_out:
        f_out.write(xml_str)

In [10]:
path = "./temp/dtsx_query/parts_basic01"
namespace = "{www.microsoft.com/SqlServer/Dts}"

# Parse the XML file
tree = ET.parse('./temp/dtsx_query/basic01.dtsx')
root = tree.getroot()

# Extract the ConnectionManagers
connection_managers = root.findall(f".//{namespace}ConnectionManagers")
variables = root.findall(f".//{namespace}Variable")
precedence_constraints = root.findall(f".//{namespace}PrecedenceConstraints")

save_part(connection_managers, "connection_managers", path)
save_part(variables, "variables", path)
save_part(precedence_constraints, "precedence_constraints", path)

executables = root.findall(f".//{namespace}Executable")
for executable in executables:

    object_data = executable.find(f".//{namespace}ObjectData")
    script_project = executable.find(f".//ScriptProject")

    if script_project is not None:
        script_project_name = script_project.attrib['Name']
        save_part(script_project, f"script_project-{script_project_name}", path)
        object_data.remove(script_project)
        object_data.append(ET.fromstring(f"<ScriptProjectExtracted Name='{script_project_name}'>{script_project_name}</ScriptProjectExtracted>"))
        
save_part(executables, "executables", path)


In [11]:
dtsx_context = ""
files = [
    "connection_managers.txt",
    "executables.txt",
    "precedence_constraints.txt",
    "variables.txt",
]
for file in files:
    with open(f"{path}/{file}", "r") as f_in:
        dtsx_context += f"{f_in.read()}\n"

print(dtsx_context)

<ns0:ConnectionManagers xmlns:ns0="www.microsoft.com/SqlServer/Dts">
    <ns0:ConnectionManager ns0:refId="Package.ConnectionManagers[ECBPPRQ121,11124.BDDELIVERY]" ns0:CreationName="OLEDB" ns0:DTSID="{F86D37AE-A474-4354-89A7-7A232A318205}" ns0:ObjectName="ECBPPRQ121,11124.BDDELIVERY">
      <ns0:ObjectData>
        <ns0:ConnectionManager ns0:ConnectRetryCount="1" ns0:ConnectRetryInterval="5" ns0:ConnectionString="Data Source=ECBPPRQ121,11124;Initial Catalog=BDDELIVERY;Provider=SQLNCLI11.1;Integrated Security=SSPI;Application Name=SSIS-Package-{F86D37AE-A474-4354-89A7-7A232A318205}ECBPPRQ121,11124.BDDELIVERY;Auto Translate=False;" />
      </ns0:ObjectData>
    </ns0:ConnectionManager>
  </ns0:ConnectionManagers>
<ns0:Executable xmlns:ns0="www.microsoft.com/SqlServer/Dts" xmlns:ns1="www.microsoft.com/sqlserver/dts/tasks/sqltask" ns0:refId="Package\Tarea Ejecutar SQL" ns0:CreationName="Microsoft.ExecuteSQLTask" ns0:Description="Tarea Ejecutar SQL" ns0:DTSID="{E950AC02-29BD-47C8-B327-7AF3

In [12]:
actual_chat = gen_ai_llm.component_memory.get_raw_memory()
# PROMPT_DIRECT = f"""{actual_chat}
# {human_prefix}: ¿Qué es un archivo DTSX? ¿Cómo es su estructura?
# {ai_prefix}: 
# """.strip()
PROMPT_DIRECT = f"""Te compartiré un fragmento de un archivo DTSX, tu tarea será entenderlo y extraer metadata del mismo.
Por cada tarea se debe extraer la siguiente información en una tabla cuyo separador es '|':
1. Orden.- El orden de precedencia de la tarea.
2. Tarea.- Es el nombre de la tarea, executable o caja.
3. Tipo de tarea.- Si es posible, identificar el tipo de tarea o caja.
4. Descripción de tarea.- Luego de ver la tarea al completo se podría generar una descripción útil.
5. Servidor de entrada.- Si lo tiene, mapear el nombre del servidor de entrada con su puerto. Un servidor de entrada puede ser un nombre de un SQL Server por ejemplo de donde vienen los datos.
6. Base de datos de entrada.- Si lo tiene, mapear el nombre de la base de datos de entrada. Usualmente la base de entrada suele estar cerca del nombre del servidor de entrada.
7. Tabla de entrada.- Si la tiene, mapear el nombre de la tabla usada desde la base de datos de entrada.
8. Campo de entrada.- Si es posible, determinar el campo o campos usados en la entrada.
9. Parametros de entrada.- Lista de todos los parámetros enviados como entrada a la tarea.
10. Procesamiento.- Esta es la parte central. Es un resumen de lo que hace la tarea, el script, código de programación o algo identificativo que sea ejecutado por la tarea. En el caso de un C# script por ejemplo es todo el código de programación, para una tarea de SQL puede ser todo el SQL Statement. 
11. Servidor de salida.- Si lo tiene, mapear el nombre del servidor de salida. Este puede ser en dónde termina la salida de la tarea.
12. Base de datos de salida.- Si lo tiene, mapear el nombre de la base de datos asociado al servidor de salida.
13. Tabla de salida.- Si la tiene, mapear el nombre de la tabla usada desde la base de datos de salida.
14. Campo de salida.- Si es posible, determinar el campo o campos usados para la salida.
15. Resultado de salida.- Lista de todos los resultados que salen de la tarea.
16. Extracteds.- Son fragmentos XML que fueron extraidos y reemplazados por una versión mínima

Por ejemplo si existe una tarea SQL llamada 'SUB-1234' que trunca datos en una tabla. Todo esto debe desembocar en un registro. En el procesamiento mapea las etiquetas xml que tengan 'Extracted' en el nombre:

Orden|Tarea|Tipo de tarea|Descripción de tarea|Servidor de entrada|Base de datos de entrada|Tabla de entrada|Campo de entrada|Parametros de entrada|Procesamiento|Servidor de salida|Base de datos de salida|Tabla de salida|Campo de salida|Resultado de salida
1|SUB-1234|Ejecutar SQL|Realiza un conteo de datos|ECBPPRQ29 11429|DQ_BI||||TRUNCATE TABLE [dbo].[Q59_DTSX_INFO]||||||ScriptProjectExtracted

El DTSX es: {dtsx_context}
"""
llm_result = gen_ai_llm.invoke(PROMPT_DIRECT, save_in_memory=True)

print(llm_result)

2024-02-14 17:45:58.684020-05:00: Invocando modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO_16k con Te compartiré un fragmento de un archivo DTSX, tu tarea será entenderlo y extraer metadata del mismo.
Por cada tarea se debe extraer la siguiente información en una tabla cuyo separador es '|':
1. Orden.- El orden de precedencia de la tarea.
2. Tarea.- Es el nombre de la tarea, executable o caja.
3. Tipo de tarea.- Si es posible, identificar el tipo de tarea o caja.
4. Descripción de tarea.- Luego de ver la tarea al completo se podría generar una descripción útil.
5. Servidor de entrada.- Si lo tiene, mapear el nombre del servidor de entrada con su puerto. Un servidor de entrada puede ser un nombre de un SQL Server por ejemplo de donde vienen los datos.
6. Base de datos de entrada.- Si lo tiene, mapear el nombre de la base de datos de entrada. Usualmente la base de entrada suele estar cerca del nombre del servidor de entrada.
7. Tabla de entrada.- Si la tiene, mapear el nombre d

La información extraída de las tareas del flujo en el archivo DTSX es la siguiente:

1. Tarea Ejecutar SQL:
   - Orden: 1
   - TaskName: Tarea Ejecutar SQL
   - Servidor Entrada: ECBPPRQ121,11124
   - Base de datos Entrada: BDDELIVERY
   - Tabla Entrada: ClientesRolerosPublicosMesBackup20231107
   - Proceso: SELECT count(*) as conteo FROM [BDDELIVERY].[UIO\ddiazpad].[ClientesRolerosPublicosMesBackup20231107] WITH(NOLOCK)

2. Tarea Script:
   - Orden: 2
   - TaskName: Tarea Script
   - Servidor Entrada: N/A
   - Base de datos Entrada: N/A
   - Tabla Entrada: N/A
   - Proceso: C# Script (ver ScriptMain.cs en el archivo)


'Assistant'

# END